# Intro to LLMstudio
In this tutorial you'll be running the proxy and the tracker so you can automatically log the usage metrics and costs in the default database (`llmstudio_mgmt.db`).

You'll learn:
1. How to connect to any provider available (VertexAI, OpenAI, etc.)
2. Make sync and async calls both with and without streaming
3. See the save logs

First things first:
* run `pip install llmstudio[proxy, tracker]`
* update your .env file with `GOOGLE_API_KEY` or `OPENAI_API_KEY`


## Start proxy and tracking servers
This step is to set the proxy and tracking servers.
You can opt out to start such servers (see `llmstudio-core` for the lightest version)

In [1]:
from pprint import pprint
from llmstudio.server import start_servers
from llmstudio_tracker.tracker import TrackingConfig
from llmstudio_proxy.provider import ProxyConfig

# start servers
start_servers(proxy=True, tracker=True)

proxy = ProxyConfig(host="0.0.0.0", port="8001")
tracking = TrackingConfig(host="0.0.0.0", port="8002")



/Users/diogo/fun/LLMstudio/libs/proxy/pyproject.toml
Running LLMstudio Proxy on http://0.0.0.0:8001 
/Users/diogo/fun/LLMstudio/libs/tracker/pyproject.toml
Running LLMstudio Tracking on http://0.0.0.0:8002 


In [18]:
from llmstudio.providers import LLM

# You can set OPENAI_API_KEY and GOOGLE_API_KEY on .env file or pass as api_key on LLM

vertexai = LLM("vertexai",
               proxy_config=proxy,
               tracking_config=tracking,
               session_id="vertexai-1")

openai = LLM("openai",
             proxy_config=proxy,
             tracking_config=tracking,
             session_id="openai-1")


Connected to LLMStudio Proxy @ 0.0.0.0:8001
Connected to LLMStudio Proxy @ 0.0.0.0:8001


# VertexAI
### Chat (non-stream)

In [3]:
response = vertexai.chat("What's your name", model="gemini-1.5-pro-latest")
print(response.chat_output)
pprint(response.metrics)

I am a large language model, so I don't have a name like a person. You can call me your helpful assistant, or just assistant! 😊  How can I help you today? 

{'cost_usd': 0.0004319,
 'input_tokens': 4,
 'inter_token_latency_s': 0.11219484465462821,
 'latency_s': 2.0175440311431885,
 'output_tokens': 41,
 'time_to_first_token_s': 1.1958141326904297,
 'tokens_per_second': 3.965217054255321,
 'total_tokens': 45}


### Chat Async (non-stream)

In [4]:
await vertexai.achat("What's your name", model="gemini-1.5-pro-latest")

ChatCompletion(id='f778fb45-8b44-4b90-9b94-b5e0029a265e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have a name. I'm a large language model, and I'm here to assist you. 😊 \n\nDo you need help with anything today? \n", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729701406, model='gemini-1.5-pro-latest', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output="I don't have a name. I'm a large language model, and I'm here to assist you. 😊 \n\nDo you need help with anything today? \n", chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='vertexai', deployment='gemini-1.5-pro-latest', timestamp=1729701406.881098, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 34, 'total_tokens': 38, 'cost_usd': 0.0003584, 'latency_s': 1.9173047542572021, 'time_to_first_token

### Chat (stream)

In [5]:
response = vertexai.chat("Write a paragfraph about space", model="gemini-1.5-pro-latest", is_stream=True)
for i, chunk in enumerate(response):
    if i%2==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, a boundless expanse of cosmic wonder, stretches out infinitely in all directions. Filled

 with swirling galaxies, shimmering nebulae, and enigmatic black holes, it is a realm of breathtaking beauty and mind-boggling mysteries. From the fiery birth of stars to the silent dance of planets around distant suns, space is a vast canvas

 upon which the universe paints its awe-inspiring story. Its sheer immensity humbles us, reminding us of our place in the grand scheme of things, while simultaneously igniting our curiosity to explore its uncharted territories and unravel its deepest secrets. 




## Metrics:
{'cost_usd': 0.0012838,
 'input_tokens': 8,
 'inter_token_latency_s': 0.1986620859666304,
 'latency_s': 3.5033671855926514,
 'output_tokens': 122,
 'time_to_first_token_s': 1.2898240089416504,
 'tokens_per_second': 3.425276131302807,
 'total_tokens': 130}


### Chat Async (stream)

In [6]:
i=0
async for chunk in await vertexai.achat("Write a paragfraph about space", model="gemini-1.5-pro-latest", is_stream=True):
    if i%20==2:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1


Space, an infinite expanse of darkness sprinkled with the brilliant light of celestial objects, continues

 to captivate humanity's imagination.  Vast distances separate swirling galaxies, each a swirling city of stars, planets, and cosmic dust.  Within these galaxies, nebulae paint the cosmos with vibrant colors, marking the birthplaces and graveyards of stars.  Black holes, enigmatic objects of immense gravity, lurk unseen, warping space and time.  Every glimpse into the depths of space unveils new wonders and mysteries, reminding us of the universe's incredible scale and our own small, yet significant, place within it. 


## Metrics:
{'cost_usd': 0.0013573,
 'input_tokens': 8,
 'inter_token_latency_s': 0.20904343778436835,
 'latency_s': 3.5030388832092285,
 'output_tokens': 129,
 'time_to_first_token_s': 1.1743888854980469,
 'tokens_per_second': 3.4255971458148577,
 'total_tokens': 137}


### Get session metrics and usage

In [7]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracking)

In [16]:
logs = tracker.get_session_logs(session_id="vertexai-1")
logs.json()[-1]


{'chat_input': 'Write a paragfraph about space',
 'chat_output': "Space, an infinite expanse of darkness sprinkled with the brilliant light of celestial objects, continues to captivate humanity's imagination.  Vast distances separate swirling galaxies, each a swirling city of stars, planets, and cosmic dust.  Within these galaxies, nebulae paint the cosmos with vibrant colors, marking the birthplaces and graveyards of stars.  Black holes, enigmatic objects of immense gravity, lurk unseen, warping space and time.  Every glimpse into the depths of space unveils new wonders and mysteries, reminding us of the universe's incredible scale and our own small, yet significant, place within it. \n",
 'session_id': 'vertexai-1',
 'context': [{'role': 'user', 'content': 'Write a paragfraph about space'}],
 'provider': 'vertexai',
 'model': 'gemini-1.5-pro-latest',
 'deployment': 'gemini-1.5-pro-latest',
 'parameters': {},
 'metrics': {'input_tokens': 8,
  'output_tokens': 129,
  'total_tokens': 13

# OpenAI
### Chat (non-stream)

In [9]:
response = openai.chat("What's your name", model="gpt-4o")
print(response.chat_output)
pprint(response.metrics)

I’m called ChatGPT. How can I assist you today?
{'cost_usd': 0.00023,
 'input_tokens': 4,
 'inter_token_latency_s': 0.013863001550946916,
 'latency_s': 0.5917928218841553,
 'output_tokens': 14,
 'time_to_first_token_s': 0.39728283882141113,
 'tokens_per_second': 25.346708248746353,
 'total_tokens': 18}


### Chat Async (non-stream)

In [10]:
await openai.achat("What's your name", model="gpt-4o")

ChatCompletion(id='9d45030f-441c-4818-9c6b-30cd576b74f1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have a personal name, but you can call me Assistant. How can I help you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729701414, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output="I don't have a personal name, but you can call me Assistant. How can I help you today?", chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729701415.330328, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 22, 'total_tokens': 26, 'cost_usd': 0.00035, 'latency_s': 0.6936230659484863, 'time_to_first_token_s': 0.5820357799530029, 'inter_token_latency_s': 0.005048545924100009, 'tokens_per_second': 33.1

### Chat (stream)

In [11]:
response = openai.chat("Write a paragfraph about space", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, the vast expanse beyond Earth's atmosphere, is a realm of endless fascination and mystery.

 It is an ever-expanding universe filled with galaxies, stars, planets, and other celestial objects,

 offering a glimpse into the origins and workings of the cosmos. From the brilliant luminosity of distant stars

 to the enigmatic allure of black holes, space captivates the human imagination and drives us to explore its

 depths. Advances in technology have allowed us to send spacecraft beyond our solar system, revealing the complexity and

 diversity of otherworldly environments. Space also presents challenges, such as extreme temperatures, radiation, and

 microgravity, which require innovative solutions for exploration. As humanity continues to study and venture into space,

 we not only seek to understand more about the universe but also about our own existence and the delicate balance

 of life on Earth.

## Metrics:
{'cost_usd': 0.002545,
 'input_tokens': 8,
 'inter_token_latenc

### Chat Async (stream)

In [12]:
i=0
async for chunk in await openai.achat("Write a paragfraph about space", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




Space, the vast and seemingly infinite expanse that surrounds our planet, beckons with its mysteries

 and wonders. It is a realm where time and distance stretch beyond human comprehension, filled with spinning galaxies

, brilliant stars, and planets waiting to be explored. From the breathtaking beauty of nebulae,

 where stars are born, to the eerie silence of the void, space captivates the imagination and challenges

 our understanding of the universe. Each discovery, whether it be a distant exoplanet or the swirling

 patterns of cosmic microwave radiation, deepens our appreciation for the complexity and fragility of existence. As

 technology propels us further into this celestial frontier, the quest to understand our place among the stars continues

 to inspire generations, forging connections across cultures and kindling a sense of wonder that is uniquely human.



## Metrics:
{'cost_usd': 9.72e-05,
 'input_tokens': 8,
 'inter_token_latency_s': 0.012189917266368866,
 'latency_

In [13]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracking)

In [15]:
logs = tracker.get_session_logs(session_id="openai-1")
logs.json()[-1]

{'chat_input': 'Write a paragfraph about space',
 'chat_output': 'Space, the vast and seemingly infinite expanse that surrounds our planet, beckons with its mysteries and wonders. It is a realm where time and distance stretch beyond human comprehension, filled with spinning galaxies, brilliant stars, and planets waiting to be explored. From the breathtaking beauty of nebulae, where stars are born, to the eerie silence of the void, space captivates the imagination and challenges our understanding of the universe. Each discovery, whether it be a distant exoplanet or the swirling patterns of cosmic microwave radiation, deepens our appreciation for the complexity and fragility of existence. As technology propels us further into this celestial frontier, the quest to understand our place among the stars continues to inspire generations, forging connections across cultures and kindling a sense of wonder that is uniquely human.',
 'session_id': 'openai-1',
 'context': [{'role': 'user', 'conten